In [ ]:
import wandb
wandb.init(project="fashion-mnist-q7")
X_train, Y_train, X_test, Y_test = load_fashion_mnist()
BEST_CONFIG = {
    "num_layers": 5,
    "hidden_size": 130,
    "batch_size": 65,
    "learning_rate": 0.001,
    "optimizer": "nadam",
    "epochs": 10
}
hidden_layers = [BEST_CONFIG["hidden_size"]] * BEST_CONFIG["num_layers"]
model = FNN(
    input_size=X_train.shape[1],
    hidden_layers=hidden_layers,
    output_size=10
)
for epoch in range(BEST_CONFIG["epochs"]):
    perm = np.random.permutation(X_train.shape[0])
    X_shuf, Y_shuf = X_train[perm], Y_train[perm]
    for i in range(0, X_shuf.shape[0], BEST_CONFIG["batch_size"]):
        xb = X_shuf[i:i+BEST_CONFIG["batch_size"]]
        yb = Y_shuf[i:i+BEST_CONFIG["batch_size"]]
        preds = model.forward(xb)
        model.backward(yb)
        model.update_parameters(
            lr=BEST_CONFIG["learning_rate"],
            optimizer=BEST_CONFIG["optimizer"],
            t=epoch+1
        )
Y_pred_test = model.forward(X_test)
y_pred = np.argmax(Y_pred_test, axis=1)
y_true = np.argmax(Y_test, axis=1)
test_accuracy = np.mean(y_pred == y_true)
wandb.log({"test_accuracy": test_accuracy})
print(f"Test Accuracy (Best Model): {test_accuracy:.4f}")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
cm = confusion_matrix(y_true, y_pred)
cm_norm = cm / cm.sum(axis=1, keepdims=True)
plt.figure(figsize=(9,7))
sns.heatmap(
    cm_norm,
    annot=True,
    fmt=".2f",
    cmap="viridis",
    xticklabels=range(10),
    yticklabels=range(10)
)
plt.title("Normalized Confusion Matrix – Best Model (Test Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.tight_layout()
wandb.log({"confusion_matrix": wandb.Image(plt)})
plt.show()
wandb.finish()